In [1]:
import pandas as pd
import numpy as np
from numpy import asarray

from PIL import Image

from numpy import expand_dims
from numpy import savez_compressed

from tensorflow.keras.models import load_model

from tqdm import tqdm
from tqdm import tqdm_notebook

import ast

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the FaceNet model
facenet_model = load_model('FaceNet_Model/facenet_keras.h5')
facenet_model.load_weights('FaceNet_Model/facenet_keras_weights.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [3]:
#Load the Dataframes with embeddings
photo_df_a = pd.read_csv('photo_info_df/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_a.csv')
photo_df_b = pd.read_csv('photo_info_df/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_b.csv')
photo_df_c = pd.read_csv('photo_info_df/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_c.csv')
photo_df_d = pd.read_csv('photo_info_df/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_d.csv')
photo_df_e = pd.read_csv('photo_info_df/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_e.csv')
photo_df_f = pd.read_csv('photo_info_df/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_f.csv')
photo_df_g = pd.read_csv('photo_info_df/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_g.csv')
photo_df_h = pd.read_csv('photo_info_df/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_h.csv')

In [4]:
#Upon further inspection row 7428 in Dataframe h is the only one that breaks in my conversion function,
#this is because nans were returned by the model, I looked into this further and found that the picture
#is just a white square, I remove this row here
photo_df_h.drop(photo_df_h.index[7428], inplace = True)


#Concat the dataframes
photo_df = pd.concat([photo_df_a, photo_df_b, photo_df_c, photo_df_d, photo_df_e, photo_df_f, photo_df_g, photo_df_h]).reset_index(drop=True)

In [5]:
#This function converts the saved string in the 'embeddings_fn' column to a numpy array

def convert_csv_to_embeddings(embedding_string):
    
    #I replace the '\n' and spaces in descending sequential order
    embedding_string = embedding_string.replace('\n', '').replace('     ', ' ').replace('    ', ' ').replace('   ', ' ').replace('  ', ' ').replace('[ ', '[').replace(' ]', ']').replace(' ', ', ')
    
    #This returns the string as an array in the proper type
    return asarray(ast.literal_eval(embedding_string)).astype('float32')

In [6]:
tqdm.pandas()
photo_df.embeddings_fn = photo_df.embeddings_fn.progress_apply(lambda x: convert_csv_to_embeddings(x))





100%|██████████| 372916/372916 [02:30<00:00, 2475.25it/s]


In [7]:
photo_df.head()

,name,dob,gender,photo_taken,age_when_taken,file_path,face_location,face_score,embeddings_fn
0,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm124825600_...,[1072.926 161.838 1214.784 303.696],1.459693,"[-0.74029577, 0.14542007, -0.5782049, -0.00768..."
1,Fred Astaire,1899-05-10,1.0,1970,71,image_data/imdb_data/01/nm0000001_rm3343756032...,[477.184 100.352 622.592 245.76 ],2.543198,"[-0.1463713, -0.5106522, -0.48219085, -0.48631..."
2,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm577153792_...,[114.96964309 114.96964309 451.68657236 451.68...,3.455579,"[0.35580358, 0.14148678, 0.11161471, -0.294302..."
3,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm946909184_...,[622.88550564 424.21750384 844.33900767 645.67...,1.872117,"[0.1044133, -0.80462444, -0.86017245, -0.59309..."
4,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm980463616_...,[1013.85900236 233.88204221 1201.5861278 42...,1.158766,"[-1.3740315, -0.807325, -0.21413553, 0.0199386..."


In [9]:
photo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372916 entries, 0 to 372915
Data columns (total 9 columns):
name              372916 non-null object
dob               372916 non-null object
gender            372916 non-null float64
photo_taken       372916 non-null int64
age_when_taken    372916 non-null int64
file_path         372916 non-null object
face_location     372916 non-null object
face_score        372916 non-null float64
embeddings_fn     372916 non-null object
dtypes: float64(2), int64(2), object(5)
memory usage: 25.6+ MB


In [15]:
photo_df.embeddings_fn[500]

array([-2.8159583e+00, -2.6450622e-01,  1.9277775e+00, -1.6459100e+00,
        5.0666231e-01,  9.2068277e-02, -4.1450985e-02,  6.7421123e-02,
       -4.0280741e-01,  9.1429427e-02,  2.8529257e-02,  8.9647895e-01,
        7.8710991e-01,  1.3718821e-01,  2.6549125e-01,  8.9681083e-01,
       -2.1329600e-01,  1.0628946e+00,  1.0834318e+00, -4.2698783e-01,
        4.8099378e-01,  2.3107894e-01, -2.1850953e-01,  1.0767412e+00,
       -3.9961949e-01,  3.4544349e-01,  2.5518230e-01,  1.2890376e+00,
        2.5534055e-01,  5.3035939e-01, -1.1170588e+00, -3.1961018e-01,
       -1.5927204e-01,  9.0936579e-02,  1.1449692e+00, -4.7476184e-01,
       -1.6302524e+00,  1.6039766e-01, -7.8691703e-01, -1.9301774e-01,
        2.5980222e-01, -1.6283334e+00, -5.0829932e-02,  9.7849631e-01,
       -7.8738636e-01, -1.3548354e+00,  1.9182414e-01, -1.5684148e+00,
       -6.5695792e-01, -9.3626916e-01, -5.2665865e-01,  1.5173119e+00,
        1.1070720e+00, -7.7063578e-01,  3.0547881e-01, -9.9846983e-01,
      